In [1]:
import pandas as pd

In [2]:
pub = pd.read_csv('./PUB_schools.csv')

In [3]:
enroll = pd.read_csv('./school_enrrollment_1516.csv')

In [4]:
# join schools and enrollment categories
pub_enroll = pd.merge(pub, enroll, left_on='CDSCode', right_on='CDS_CODE', how='left')
pub_enroll.head()

,CDSCode,StatusType,District,School,Street,StreetAbr,City,Zip,State,County,...,GR_7,GR_8,UNGR_ELM,GR_9,GR_10,GR_11,GR_12,UNGR_SEC,ENR_TOTAL,ADULT
0,1100170000000,Active,Alameda County Office of Education,NaN,313 West Winton Avenue,313 West Winton Ave.,Hayward,94544-1136,CA,Alameda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1100170109835,Closed,Alameda County Office of Education,FAME Public Charter,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,94560-5359,CA,Alameda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1100170112607,Active,Alameda County Office of Education,Envision Academy for Arts & Technology,1515 Webster Street,1515 Webster St.,Oakland,94612-3355,CA,Alameda,...,0.0,0.0,0.0,21.0,23.0,20.0,24.0,0.0,88.0,0.0
3,1100170112607,Active,Alameda County Office of Education,Envision Academy for Arts & Technology,1515 Webster Street,1515 Webster St.,Oakland,94612-3355,CA,Alameda,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0
4,1100170112607,Active,Alameda County Office of Education,Envision Academy for Arts & Technology,1515 Webster Street,1515 Webster St.,Oakland,94612-3355,CA,Alameda,...,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,5.0,0.0


In [5]:
# check how the join did
print(len(enroll[(enroll['COUNTY'] == 'Alameda') | (enroll['COUNTY'] == 'Contra Costa') | (enroll['COUNTY'] == 'Marin') |
                 (enroll['COUNTY'] == 'Napa') | (enroll['COUNTY'] == 'Solano') | (enroll['COUNTY'] == 'Sonoma') |
                 (enroll['COUNTY'] == 'San Mateo') | (enroll['COUNTY'] == 'San Francisco') | (enroll['COUNTY'] == 'Santa Clara')]))
print(len(pub_enroll[pub_enroll['ADULT'].notnull()]))

25270
24663


In [6]:
# remove closed schools
print(len(pub_enroll))
pub_enroll = pub_enroll[pub_enroll['StatusType'] != 'Closed']
print(len(pub_enroll))

26291
24982


In [7]:
pub_enroll.to_csv('./PUB_schools_enroll.csv')

In [8]:
prv_enroll = pd.read_csv('./PRV_schools_enroll.csv')

In [9]:
# geocode both private and public schools (ArcMap)
# join them to TAZ1454 with "contains" (ArcMap)

In [10]:
taz_pub = pd.read_csv('taz_pub.csv')

C:\Users\etheocharides\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (17,18,20,24,33,34,38,39,40,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# check for duplicate schools from "contains" join
print(len(taz_pub[taz_pub['JOIN_FID'] != -1]))
print(len(taz_pub.drop_duplicates('JOIN_FID', False)))

24926
24926


In [12]:
# check if we lost any schools in the geocoding
print(len(pub_enroll))
print(len(taz_pub[taz_pub['JOIN_FID'] != -1]))

24982
24926


In [13]:
# sumamrize by TAZ
taz_pub_grp = taz_pub.groupby(['TAZ1454']).sum()

In [14]:
# add a column for HS enrollment
# ungraded secondary and adult NOT included
taz_pub_grp['HS_pub'] = (taz_pub_grp['USER_GR_9'] + taz_pub_grp['USER_GR_10'] + taz_pub_grp['USER_GR_11'] 
                        + taz_pub_grp['USER_GR_12'])

In [15]:
taz_prv = pd.read_csv('taz_prv.csv')

In [16]:
# check for duplicate schools
print(len(taz_prv[taz_prv['JOIN_FID'] != -1]))
print(len(taz_prv.drop_duplicates('JOIN_FID', False)))

724
724


In [17]:
# check if we lost any schools in the geocoding
print(len(prv_enroll))
print(len(taz_prv[taz_prv['JOIN_FID'] != -1]))

725
724


In [18]:
# sumamrize by TAZ
taz_prv_grp = taz_prv.groupby(['TAZ1454']).sum()

In [19]:
# add a column for HS enrollment
# ungraded secondary NOT included
taz_prv_grp['HS_prv'] = (taz_prv_grp['USER_Grade_9_Enrollment'] + taz_prv_grp['USER_Grade_10_Enrollment']  
                         + taz_prv_grp['USER_Grade_11_Enrollment'] + taz_prv_grp['USER_Grade_12_Enrollment'])

In [20]:
# join public and private together, create HS column
taz_enroll = taz_pub_grp.join(taz_prv_grp, lsuffix='pub', rsuffix='prv')
taz_enroll['HS'] = taz_enroll['HS_pub'] + taz_enroll['HS_prv']
taz_enroll.head()

,OBJECTIDpub,Join_Countpub,TARGET_FIDpub,JOIN_FIDpub,SUPERDpub,Shape__Arepub,Shape__Lenpub,diffpub,Scorepub,URLpub,...,USER_Grade_10_Enrollment,USER_Grade_11_Enrollment,USER_Grade_12_Enrollment,USER_Ungraded_Elementary_Enr_103,USER_Ungraded_Secondary_Enro_104,USER_Total_Enrollment,Shape_Lengthprv,Shape_Areaprv,HS_prv,HS
TAZ1454,,,,,,,,,,,,,,,,,,,,,
1,0.0,0,1338,-1,1,0.0,0.0,11,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1206.100233,82010.140753,0.0,0.0
2,0.0,0,1415,-1,1,0.0,0.0,262,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1433.237770,125828.768964,0.0,0.0
3,0.0,0,1339,-1,1,0.0,0.0,156,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,300.0,982.566396,59472.890690,0.0,0.0
4,0.0,0,1416,-1,1,0.0,0.0,287,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1283.522513,77864.452142,0.0,0.0
5,0.0,0,1340,-1,1,0.0,0.0,430,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1954.837387,213322.683012,0.0,0.0


In [21]:
# check final enrollment
pub_enroll['HSENR'] = pub_enroll['GR_9'] + pub_enroll['GR_10'] + pub_enroll['GR_11'] + pub_enroll['GR_12']
print(pub_enroll['HSENR'].sum())
print(taz_enroll['HS_pub'].sum())
prv_enroll['HSENR'] = (prv_enroll['Grade 9 Enrollment'] + prv_enroll['Grade 10 Enrollment'] + prv_enroll['Grade 11 Enrollment'] 
                      + prv_enroll['Grade 12 Enrollment'])
print(prv_enroll['HSENR'].sum())
print(taz_enroll['HS_prv'].sum())

307024.0
306282.0
40652
40652.0


In [22]:
# check final enrollment
print(pub_enroll['ENR_TOTAL'].sum() + prv_enroll['Total Enrollment'].sum())
print(taz_enroll['USER_ENR_TOTAL'].sum() + taz_enroll['USER_Total_Enrollment'].sum())

1156995.0
1155245.0


In [23]:
taz_enroll_clip = taz_enroll[['HS']]
taz_enroll_clip.to_csv('2015_HS_enrollment_TAZ1454.csv')